In [4]:
import os
import dotenv
dotenv.load_dotenv()

from openai import OpenAI

In [ ]:
client = OpenAI(api_key=os.getenv("openai"))

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "test"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Write a long poem?"
        }
      ]
    }
  ],
  temperature=1,
  max_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  response_format={
    "type": "text"
  },
  stream=True
)
for chunk in response:
    print(chunk.choices[0].delta.content, end= '')

In [ ]:
import psycopg2
conn = psycopg2.connect(
    dbname="startup_database",
    user="postgres",
    password=os.getenv("DB_PASSWORD"),
    host="localhost",
    port="5432"
)
cur = conn.cursor()
cur.execute("SELECT * FROM crunchbase ORDER BY CB_Rank LIMIT 200")

records = cur.fetchall()

In [ ]:
len(records)

In [14]:
from core.database import models

In [ ]:
class_keys=list(models.crunchbase_fields.keys())
company_name_index=class_keys.index("Company_Name")
company_description_index=class_keys.index("Short_Description")
companies_context_prompt=""
for i, record in enumerate(records[0:20]):
    companies_context_prompt+="ID: "+str(i)+" | Name: "+record[company_name_index]+" | Description: "+record[company_description_index]+"\n"
    # print("ID: "+str(i)+" | Name: "+record[company_name_index]+" | Description: "+record[company_description_index])
print(len(companies_context_prompt))

In [ ]:
client = OpenAI(api_key=os.getenv("openai"))

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are to give a rating 0-5 for a each company given to you. Format your response as follows 'ID: 0 | Rating: 5\nID: 1 | Rating: 4\n...' where 0 is the ID of the company and 5 is the rating you give the company."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": companies_context_prompt
        }
      ]
    }
  ],
  temperature=1,
  max_tokens=4048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  response_format={
    "type": "text"
  },
  stream=True
)
response_str=""
temp_string=""
rating_list=[]
for chunk in response:
    if chunk.choices[0].delta.content!=None:
        response_str+=chunk.choices[0].delta.content
        temp_string+=chunk.choices[0].delta.content

        if '\n' in temp_string:
            # print(temp_string)
            rating_list.append(temp_string.split("Rating:")[-1].strip())
            print(rating_list[-1])
            temp_string=""



        # print(chunk.choices[0].delta.content, end= '')
    else:
        # print(temp_string)
        rating_list.append(temp_string.split("Rating:".strip()))

        print(rating_list[-1])


In [2]:
from dotenv import load_dotenv
load_dotenv()
from core.ai import providers

In [4]:
%load_ext autoreload
%autoreload 2

In [ ]:
providers.ModelSelector.llama70b.model_provider

In [5]:
test=providers.AIModel("llama70b","test")

In [ ]:
test.model_provider

In [ ]:
print(providers.ModelSelector.llama70b)

In [ ]:
test_dict={"1":"test",2:"test2"}
# remove key "1" from test_dict
test_dict.pop("1")
test_dict

In [ ]:
prompt=[
    {
        "role": "system",
        "content":  "You are to give a rating 0-5 for a each company given to you. Format your response as follows 'ID: 0 | Thoughts on company in relation to user's message | Rating: 5\nID: 1 | Thoughts on company in relation to user's message | Rating: 4\n...' where 0 is the ID of the company and 5 is the rating you give the company." 
    },
    {
        "role": "user",
        "content": "Test"
    },
]

